In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Load the dataset into a Pandas dataframe
students_df = pd.read_csv('../data/dataset.csv')

In [ ]:
# set display width to a larger number
pd.set_option('display.width', 1000)

# Check the shape of the dataset
print("\nDataset shape:", students_df.shape)

# Check the data types of the variables
print("\nData types:")
print(students_df.dtypes)

# Check the first few rows of the dataset
print("\nFirst few rows:\n", students_df.head())

# Calculate the summary statistics for all columns
summary = students_df.describe().transpose().round(2)
summary = summary.rename(columns={'50%': 'median', 'std': 'sd'}) # Change 50% to median

# Add columns for the number of missing values and the completeness rate
summary['n_missing'] = students_df.isnull().sum()
summary['complete_rate'] = 1 - (students_df.isnull().sum() / len(students_df))

# Print the summary statistics
print("\nSummary statistics\n",
      summary[['mean', 'sd', 'min', '25%', 'median', '75%', 'max', 'n_missing', 'complete_rate']])


In [ ]:
# Display number of duplicated rows
print("Number of duplicated rows: ",students_df.duplicated().sum())

# Display rows with duplicates
print(students_df[students_df.duplicated()])

In [ ]:
# Get no of Unique values in every column
for col in students_df.columns:
    if len(students_df[col].unique()) < 10:
        print(f"{col} has {len(students_df[col].unique())} unique values; Repeated values are {sorted(students_df[col].unique())}")
    else:
        print(f"{col} has {len(students_df[col].unique())} values")

In [ ]:
# Defining categorical and quantitative columns based on the above checks and metadata description
categorical_cols = ["Marital status", "Application mode", "Application order", "Course", "Daytime/evening attendance",
                    "Previous qualification", "Nacionality", "Mother's qualification", "Father's qualification", "Mother's occupation",
                    "Father's occupation", "Displaced", "Educational special needs", "Debtor", "Tuition fees up to date", "Gender",
                    "Scholarship holder", "International", "Target"]

quantitative_cols = ['Curricular units 1st sem (credited)', 'Curricular units 1st sem (enrolled)', 'Curricular units 1st sem (evaluations)',
                  'Curricular units 1st sem (approved)', 'Curricular units 1st sem (grade)', 'Curricular units 1st sem (without evaluations)',
                  'Curricular units 2nd sem (credited)', 'Curricular units 2nd sem (enrolled)', 'Curricular units 2nd sem (evaluations)',
                  'Curricular units 2nd sem (approved)', 'Curricular units 2nd sem (grade)', 'Curricular units 2nd sem (without evaluations)',
                  'Age at enrollment', 'Inflation rate', 'GDP', 'Unemployment rate']

In [ ]:
# loop through categorical columns and create barplot for each
fig, axes = plt.subplots(nrows=5, ncols=4, figsize=(20, 25))
for i, col in enumerate(categorical_cols):
    row_idx = i // 4
    col_idx = i % 4
    vc = students_df[col].value_counts(normalize=True)
    vc = vc.sort_index() # sort by category name
    vc.plot(kind="bar", ax=axes[row_idx, col_idx], rot=0)
    axes[row_idx, col_idx].set_title(col)
    for tick in axes[row_idx, col_idx].get_xticklabels():
        tick.set_rotation(90)
    for tick in axes[row_idx, col_idx].get_yticklabels():
        tick.set_visible(False)
fig.tight_layout()
plt.show()

In [ ]:
# Loop through categorical columns and create frequency table for each
for col in categorical_cols:
    print(f"Frequency Table for {col}:")
    print(students_df[col].value_counts(normalize=True))
    print("\n")


In [ ]:
# Plot histograms for all quantitative columns
students_df[quantitative_cols].hist(figsize=(20, 20));

In [ ]:
# Numerical Summaries for quantitative variables
summary_df = students_df[quantitative_cols].describe().loc[['mean', 'std', 'min', '25%', '50%', '75%', 'max']]
summary_df.loc['skew'] = students_df[quantitative_cols].skew()

summary_df

In [ ]:
# Show numerical summaries for columns of interest i.e. Inflation rate and GDP
cols_of_interest = ['Inflation rate', 'GDP']
summary_df2 = students_df[cols_of_interest].describe().loc[['mean', 'std', 'min', '25%', '50%', '75%', 'max']]
summary_df2.loc['skew'] = students_df[cols_of_interest].skew()
print(summary_df2)

In [ ]:
# Show skew values for higher skew variables
print("Variables with higher skew values:")
for col in quantitative_cols:
    if abs(summary_df.loc['skew', col]) > 1:
        print(f"\t{col}: {summary_df.loc['skew', col]}")

In [ ]:
# Duplicate the data
students_df_cleaned = students_df.copy()


In [ ]:

# import library to calculate mode
from scipy.stats import mode

In [ ]:
# replace 0 values with NaN
students_df_cleaned['Application order'].replace(0, np.nan, inplace=True)

In [ ]:
# impute missing values with mode
students_df_cleaned['Application order'].fillna(mode_value, inplace=True)

In [ ]:
# Define the minimum frequency for combining categories
min_freq = 150

In [ ]:
# Combine categories with low frequency
combined_cats_dict = {}
for col in categorical_cols:
  # Check if the variable is not binary
  if len(students_df_cleaned[col].unique()) > 2:
    # Count the frequency of each category
    counts = students_df_cleaned[col].value_counts()
    # Identify the categories with low frequency
    low_freq_cats = counts[counts < min_freq].index.tolist()
    # Only combine categories if at least 2 categories have frequency less than min_freq
    if len(low_freq_cats) >= 2:
        # Combine the low frequency categories into a single "Other" category represented as -1
        students_df_cleaned[col] = students_df_cleaned[col].apply(lambda x: -1 if x in low_freq_cats else x)
        # Identify the categories that were combined
        combined_cats_dict[col] = ', '.join(str(cat) for cat in low_freq_cats)
        other_percent = len(students_df_cleaned[students_df_cleaned[col] == -1]) / len(students_df) * 100
        #print(f"Categories combined for {col}: {combined_cats_dict[col]} \t totalling: {other_percent:.2f}%")
        print(f"For {col} variable, -1 (Other) constitutes {other_percent:.2f}% & combines categories {combined_cats_dict[col]}")

print("\n")

In [ ]:
# Loop through categorical columns and create frequency table for each
for col in categorical_cols:
    # Create a frequency table for the current categorical column
    freq_table = students_df_cleaned[col].value_counts(normalize=True).mul(100).round(2).astype(str) + '%'
    print(f"Frequency Table for {col}:")
    print(freq_table)


In [ ]:
# Change Column Name in dataset
students_df_cleaned = students_df_cleaned.rename(columns={'Nacionality': 'Nationality'})

# Update the column name in the categorical_cols list
categorical_cols[categorical_cols.index('Nacionality')] = 'Nationality'

In [ ]:
# Identify columns with high skewness
high_skew_cols = summary_df.columns[(summary_df.loc['skew'] > 1) | (summary_df.loc['skew'] < -1)]

# Duplicate the dataset
students_df_normalised = students_df_cleaned.copy()

# Normalize data using log transformation
students_df_normalised[high_skew_cols] = np.sqrt(students_df_normalised[high_skew_cols])

# Alternatively, we can also use other normalization techniques
#students_df_cleaned[high_skew_cols] = np.log(students_df_cleaned[high_skew_cols])

# Numerical Summaries for quantitative variables
summary_df_normalised = students_df_normalised[quantitative_cols].describe().loc[['mean', 'std', 'min', '25%', '50%', '75%', 'max']]
summary_df_normalised.loc['skew'] = students_df_normalised[quantitative_cols].skew()

# Stack the two dataframes vertically with a separator
separator = pd.DataFrame([['----'] * len(quantitative_cols)], columns=quantitative_cols)
stacked_summary = pd.concat([summary_df, separator, summary_df_normalised], keys=['Original Summary', '----', 'Normalised Summary'])

stacked_summary

In [ ]:
# Make a copy of the original normalized dataset
students_df_normalised_no_outliers = students_df_normalised.copy()

# Calculate the IQR for each numerical column
Q1 = students_df_normalised[quantitative_cols].quantile(0.25)
Q3 = students_df_normalised[quantitative_cols].quantile(0.75)
IQR = Q3 - Q1

# Define the upper and lower bounds for outlier removal
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Identify and remove outliers
outliers = ((students_df_normalised[quantitative_cols] < lower_bound) | (students_df_normalised[quantitative_cols] > upper_bound)).any(axis=1)
students_df_normalised_no_outliers = students_df_normalised[~outliers]

In [ ]:
def corr_plot(df, title, threshold = 0.5):
  # Compute the correlation matrix on cleaned dataset
  corr_matrix = df.corr()

  # Create a mask to display only the upper triangle of the heatmap
  mask = np.triu(np.ones_like(corr_matrix, dtype=bool))

  # Create a heatmap with the high correlation variables annotated
  plt.figure(figsize=(10,8))
  sns.heatmap(corr_matrix[(corr_matrix > threshold) | (corr_matrix < -threshold)],
              cmap="coolwarm", annot=True, annot_kws={"size": 7}, mask=mask)
  title = "Heatmap showing correlation in " + title + " variables"
  plt.title(title, fontsize=18)
  plt.show()

  return corr_matrix

# Convert Target to numerical
students_df_cleaned["Target"].replace('Dropout', 0, inplace=True)
students_df_cleaned["Target"].replace('Graduate', 1, inplace=True)
students_df_cleaned["Target"].replace('Enrolled', 2, inplace=True)


# Demographic Features
demographic_df = students_df_cleaned[["Marital status", "Nationality", "Displaced",
                                      "Gender", "Age at enrollment", "International",
                                      "Target"]]

# Socio Economic Features
socioeconomic_df = students_df_cleaned[["Mother's qualification", "Father's qualification",
                                       "Mother's occupation", "Father's occupation",
                                       "Educational special needs", "Debtor",
                                       "Tuition fees up to date", "Scholarship holder",
                                       "Target"]]

# Macro and Enrollment Features
macroenrollment_df = students_df_cleaned[['Unemployment rate', 'Inflation rate', 'GDP', 'Application mode',
                                          'Application order', 'Course', 'Daytime/evening attendance',
                                          'Previous qualification', 'Target']]

# Academic Features
academic_df = students_df_cleaned[['Curricular units 1st sem (credited)', 'Curricular units 1st sem (enrolled)',
                                         'Curricular units 1st sem (evaluations)', 'Curricular units 1st sem (approved)',
                                         'Curricular units 1st sem (grade)', 'Curricular units 1st sem (without evaluations)',
                                         'Curricular units 2nd sem (credited)', 'Curricular units 2nd sem (enrolled)',
                                         'Curricular units 2nd sem (evaluations)', 'Curricular units 2nd sem (approved)',
                                         'Curricular units 2nd sem (grade)', 'Curricular units 2nd sem (without evaluations)',
                                         'Target']]

# Combining results into one
corr_df = []

# Plot the correlation heatmaps for each group with correlation threshold 0
corr_df.append(corr_plot(demographic_df, "Demographic", 0))
corr_df.append(corr_plot(socioeconomic_df, "Socio-Economic", 0))
corr_df.append(corr_plot(macroenrollment_df, "Macro-Economic and Enrollment", 0))
corr_df.append(corr_plot(academic_df, "Academic", 0))

In [ ]:
# Define features to remove
features_to_remove = ["Curricular units 1st sem (credited)", "Curricular units 1st sem (enrolled)",
                      "Curricular units 1st sem (evaluations)", "Curricular units 1st sem (approved)",
                      "Curricular units 1st sem (grade)", "Curricular units 2nd sem (approved)", "Nationality"]

# remove the specified columns from the dataframe
students_df_cleaned = students_df_cleaned.drop(features_to_remove, axis=1)

# remove columns from categorical columns
categorical_cols = [col for col in categorical_cols if col not in features_to_remove]

# remove columns from quantitative columns
quantitative_cols = [col for col in quantitative_cols if col not in features_to_remove]

In [ ]:
'''
# Create a copy of the original dataset
students_df_encoded = students_df_cleaned.copy()

# Convert the Target variable to numerical representation
students_df_encoded['Target'] = students_df_encoded['Target'].astype('category').cat.codes

# Compute the correlation matrix
corr_matrix = students_df_encoded.corr()
'''
# Compute the correlation matrix on cleaned dataset
corr_matrix = students_df_cleaned.corr()

# Set the threshold for high correlation
threshold = 0

# Create a mask to display only the upper triangle of the heatmap
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))

# Create a heatmap with the high correlation variables annotated
plt.figure(figsize=(12,8))
sns.heatmap(corr_matrix[(corr_matrix > threshold) | (corr_matrix < -threshold)],
            cmap="coolwarm", annot=True, annot_kws={"size": 6}, mask=mask)

# Add lines connecting the highly correlated variables
'''
for i in range(len(corr_matrix.columns)):
    for j in range(i):
        if abs(corr_matrix.iloc[i, j]) > threshold:
            plt.axhline(y=i+0.5, color='grey', linestyle='dotted', alpha=0.1)
            plt.axvline(x=j+0.5, color='grey', linestyle='dotted', alpha=0.1)
'''

plt.title("Correlation Matrix of All Remaining Variables", fontsize=18)
plt.show()

In [ ]:
# Store high correlations in a list of tuples and sort it by absolute value of the correlation coefficient
high_corr = []
for i in range(len(corr_matrix.columns)):
    for j in range(i):
        if abs(corr_matrix.iloc[i, j]) > threshold:
            high_corr.append((corr_matrix.columns[i], corr_matrix.columns[j], abs(corr_matrix.iloc[i, j])))
high_corr.sort(key=lambda x: x[2], reverse=True)

# Print top 10 high correlations in sorted order
for c1, c2, corr in high_corr[:10]:
    print(f"{c1} - {c2}: {round(corr,2)}")

In [ ]:
# Removing enrolled from the Target variable
students_df_cleaned = students_df_cleaned[students_df_cleaned.Target != 2]

# Check the distribution of the "Target" variable
sns.countplot(x='Target', data=students_df_cleaned)
plt.show()

In [ ]:
# Frequency table for Target
print("Target distribution")
target_counts = students_df_cleaned['Target'].value_counts()
print("0\tDropout \t", target_counts[0])
print("1\tGraduate\t", target_counts[1])

In [ ]:
# Create a 9x4 grid of subplots for the bar charts
fig, axes = plt.subplots(nrows=9, ncols=4, figsize=(20,45))

# Select all categorical columns except Target
selected_cols = [col for col in categorical_cols if col != "Target"]

# Loop through the selected columns and create a bar chart for each one
for i in range(len(selected_cols)):
    row1 = i//2
    col1 = (i % 2) * 2
    col2 = col1 + 1
    counts = students_df_cleaned.groupby([selected_cols[i], 'Target'])['Course'].count().unstack()
    counts_norm = counts.apply(lambda x: x/x.sum(), axis=1)
    counts.plot(kind='bar', stacked=True, ax=axes[row1][col1])
    axes[row1][col1].set_title(selected_cols[i] + " Count", fontsize=12)
    axes[row1][col1].set_xlabel('')
    axes[row1][col1].set_ylabel('Frequency')
    axes[row1][col1].legend(title='Target')

    counts_norm.plot(kind='bar', stacked=True, ax=axes[row1][col2])
    axes[row1][col2].set_title(selected_cols[i] + " Proportion", fontsize=12)
    axes[row1][col2].set_xlabel('')
    axes[row1][col2].set_ylabel('Proportion')
    axes[row1][col2].legend(title='Target')


# Adjust the spacing between subplots and display the figure
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Create a 2x9 grid of subplots for the bar charts (since we only want to show proportions)
fig, axes = plt.subplots(nrows=3, ncols=5, figsize=(25,12))

# Select all categorical columns except Target
selected_cols = [col for col in categorical_cols if col not in ["Target", "International", "Educational special needs"]]

# Loop through the selected columns and create a bar chart for proportion in each subplot
for i in range(len(selected_cols)):
    row = i // 5
    col = i % 5
    counts = students_df_cleaned.groupby([selected_cols[i], 'Target'])['Course'].count().unstack()
    counts_norm = counts.apply(lambda x: x/x.sum(), axis=1)
    counts_norm.plot(kind='bar', stacked=True, ax=axes[row][col])
    axes[row][col].set_title(selected_cols[i] + " Proportion", fontsize=18)
    axes[row][col].set_xlabel('')
    axes[row][col].set_ylabel('Proportion')
    axes[row][col].legend(title='Target')

# Adjust the spacing between subplots and display the figure
plt.tight_layout()
plt.show()

In [ ]:
# Loop through selected columns
for col in selected_cols:

    # Group data by column and Target, then unstack
    freq_df = students_df_cleaned.groupby([col, 'Target']).size().unstack()

    # Rename columns and reset index
    freq_df.columns = ['Dropout', 'Graduate']
    freq_df = freq_df.reset_index()

    # Compute total and percentage for each Target
    freq_df['Total'] = freq_df['Dropout'] + freq_df['Graduate']
    freq_df['Percent Dropout'] = (freq_df['Dropout'] / freq_df['Total'] * 100).round(2).fillna('-').astype(str) + '%'
    freq_df['Percent Graduate'] = (freq_df['Graduate'] / freq_df['Total'] * 100).round(2).fillna('-').astype(str) + '%'

    # Rearrange and rename columns, fill NaN values with '-'
    freq_df = freq_df[[col, 'Graduate', 'Percent Graduate', 'Dropout', 'Percent Dropout']]
    freq_df.columns = [col, 'Graduate', 'Percent', 'Dropout', 'Percent']
    freq_df = freq_df.fillna('-')

    # Print the resulting table
    print(freq_df.to_string(index=False), "\n\n")

In [ ]:
# Create a 4x4 grid of subplots for the boxplots
fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(20,8))

# Loop through the quantitative columns and create a boxplot for each one
for i in range(len(quantitative_cols)):
    row = i // 5
    col = i % 5
    sns.boxplot(x='Target', y=quantitative_cols[i], data=students_df_cleaned, ax=axes[row][col])

# Adjust the spacing between subplots and display the figure
plt.tight_layout()
plt.show()

In [ ]:
# Group the data by 'Target' and compute the numerical summary
num_summary = students_df_cleaned.groupby('Target')[quantitative_cols].describe().T

# Customize the output
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

# Print the numerical summary
num_summary

In [ ]:
# Check the distribution of the "Scholarship holder" variable
sns.countplot(x='Scholarship holder', data=students_df_cleaned)
plt.show()

In [ ]:
# Frequency table for Scholarship holder
students_df_cleaned['Scholarship holder'].value_counts()

In [ ]:
# Create a 4x4 grid of subplots for the boxplots
fig, axes = plt.subplots(nrows=4, ncols=4, figsize=(20,20))

# Loop through the quantitative columns and create a boxplot for each one
for i in range(len(quantitative_cols)):
    row = i // 4
    col = i % 4
    sns.boxplot(x='Scholarship holder', y=quantitative_cols[i], data=students_df_cleaned, ax=axes[row][col])

# Adjust the spacing between subplots and display the figure
plt.tight_layout()
plt.show()

In [ ]:
# Group the data by 'Scholarship holder' and compute the numerical summary
num_summary = students_df_cleaned.groupby('Scholarship holder')[quantitative_cols].describe().T

# Customize the output
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

# Print the numerical summary
print(num_summary)

In [ ]:
# Create a 9x4 grid of subplots for the bar charts
fig, axes = plt.subplots(nrows=9, ncols=4, figsize=(20,45))

# Select all categorical columns except Scholarship holder
selected_cols = [col for col in categorical_cols if col != "Scholarship holder"]

# Loop through the categorical columns and create a bar chart for each one
for i in range(len(selected_cols)):
    row1 = i//2
    col1 = (i % 2) * 2
    col2 = col1 + 1
    counts = students_df_cleaned.groupby([selected_cols[i], 'Scholarship holder'])['Course'].count().unstack()
    counts_norm = counts.apply(lambda x: x/x.sum(), axis=1)
    counts.plot(kind='bar', stacked=True, ax=axes[row1][col1])
    axes[row1][col1].set_title(selected_cols[i] + " Count", fontsize=12)
    axes[row1][col1].set_xlabel('')
    axes[row1][col1].set_ylabel('Frequency')
    axes[row1][col1].legend(title='Scholarship holder')

    counts_norm.plot(kind='bar', stacked=True, ax=axes[row1][col2])
    axes[row1][col2].set_title(selected_cols[i] + " Proportion", fontsize=12)
    axes[row1][col2].set_xlabel('')
    axes[row1][col2].set_ylabel('Proportion')
    axes[row1][col2].legend(title='Scholarship holder')


# Adjust the spacing between subplots and display the figure
plt.tight_layout()
plt.show()

In [ ]:
# Loop through categorical columns
for col in selected_cols:

    # Group data by column and Scholarship holder, then unstack
    freq_df = students_df_cleaned.groupby([col, 'Scholarship holder']).size().unstack()

    # Rename columns and reset index
    freq_df.columns = ['No Scholarship', 'Scholarship']
    freq_df = freq_df.reset_index()

    # Compute total and percentage for each Scholarship holder
    freq_df['Total'] = freq_df['No Scholarship'] + freq_df['Scholarship']
    freq_df['Percent No Scholarship'] = (freq_df['No Scholarship'] / freq_df['Total'] * 100).round(2).fillna('-').astype(str) + '%'
    freq_df['Percent Scholarship'] = (freq_df['Scholarship'] / freq_df['Total'] * 100).round(2).fillna('-').astype(str) + '%'

    # Rearrange and rename columns, fill NaN values with '-'
    freq_df = freq_df[[col, 'Scholarship', 'Percent Scholarship', 'No Scholarship', 'Percent No Scholarship']]
    freq_df.columns = [col, 'Scholarship', 'Percent', 'No Scholarship', 'Percent']
    freq_df = freq_df.fillna('-')

    # Print the resulting table
    print(freq_df.to_string(index=False), "\n\n")

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Create a copy of the original dataset
num_vars = students_df_cleaned[quantitative_cols].copy()

# Standardize the data
scaler = StandardScaler()
num_vars_scaled = scaler.fit_transform(num_vars)
num_vars_scaled = pd.DataFrame(num_vars_scaled, columns=quantitative_cols)

# Set up a range of k values to try
k_values = range(1, 11)

# Calculate the inertia for each value of k
inertias = []
for k in k_values:
    model = KMeans(n_clusters=k, random_state=42,  n_init=10)
    model.fit(num_vars)
    inertias.append(model.inertia_)

# Visualize the elbow plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,6))

ax1.plot(k_values, inertias, 'bo-')
ax1.set_xlabel('Number of Clusters (k)')
ax1.set_ylabel('Inertia')
ax1.set_title('Elbow Plot')

# Calculate the inertia for each value of k
inertias = []
for k in k_values:
    model = KMeans(n_clusters=k, random_state=42,  n_init=10)
    model.fit(num_vars_scaled)
    inertias.append(model.inertia_)

# Visualize the elbow plot
ax2.plot(k_values, inertias, 'bo-')
ax2.set_xlabel('Number of Clusters (k)')
ax2.set_ylabel('Inertia')
ax2.set_title('Elbow Plot (with Scaled Vars)')

plt.show()

In [ ]:
# Set the optimal k values based on the elbow methods above
k_optimal_unscaled = 3
k_optimal_scaled = 3

# Build the unscaled quantitative variables model
model_unscaled = KMeans(n_clusters=k_optimal_unscaled, random_state=42, n_init=10)
model_unscaled.fit(num_vars)

# Build the scaled quantitative variables model
model_scaled = KMeans(n_clusters=k_optimal_scaled, random_state=42, n_init=10)
model_scaled.fit(num_vars_scaled)

# Get the cluster labels for each data point
labels_unscaled = model_unscaled.labels_
labels_scaled = model_scaled.labels_

# Visualize the clusters using scatter points
fig, (ax2, ax1) = plt.subplots(1, 2, figsize=(12, 5))

# Unscaled Variables Model
ax2.scatter(num_vars_scaled.iloc[:, num_vars.columns.get_loc("Age at enrollment")], num_vars_scaled.iloc[:, num_vars.columns.get_loc("Curricular units 2nd sem (credited)")], c=labels_unscaled)
centers = model_unscaled.cluster_centers_
ax2.scatter(centers[:, 0], centers[:, 1], c='red', marker='x')
ax2.set_title('K-Means Clustering (Unscaled Variables, k=3)', fontsize=14)
ax2.set_xlabel('Age at enrollment', fontsize=12)
ax2.set_ylabel('Curricular units 2nd sem (credited)', fontsize=12)

# Scaled Variables Model
ax1.scatter(num_vars_scaled.iloc[:, num_vars.columns.get_loc("Age at enrollment")], num_vars_scaled.iloc[:, num_vars.columns.get_loc("Curricular units 2nd sem (credited)")], c=labels_scaled)
centers = model_scaled.cluster_centers_
ax1.scatter(centers[:, 0], centers[:, 1], c='red', marker='x')
ax1.set_title('K-Means Clustering (Scaled Variables, k=3)', fontsize=14)
ax1.set_xlabel('Age at enrollment (Scaled)', fontsize=12)
ax1.set_ylabel('Curricular units 2nd sem (credited) (Scaled)', fontsize=12)

plt.show()

In [ ]:
# Import necessary libraries
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

## Split the dataset into quantitative variables only

# Using all columns except Target, we need 6 PCs to cummulatively explain 80% of the variance
#num_vars = students_df.select_dtypes(include=np.number)

# Using predefined quantitative columns, we need 3 PCs to cummulatively explain 80% of the variance
num_vars = students_df_cleaned[quantitative_cols]

# Standardize the data
scaler = StandardScaler()
num_vars_scaled = scaler.fit_transform(num_vars)

# Perform PCA on non-scaled variables
pca = PCA()
pca.fit(num_vars)

# Get the eigenvalues and eigenvectors
eigenvalues = pca.explained_variance_
eigenvectors = pca.components_

# Get the explained variance ratio for each principal component
explained_variance_ratio = pca.explained_variance_ratio_

# Create a figure with two subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
# Create a bar graph of the explained variance ratio for each principal component
ax1.bar(range(1, len(explained_variance_ratio)+1), explained_variance_ratio)
ax1.set_xticks(range(1, len(explained_variance_ratio)+1))
ax1.set_xlabel('Principal Component')
ax1.set_ylabel('Explained Variance Ratio')
ax1.set_title('Explained Variance Ratio for each Principal Component')

# Plot the cumulative explained variance ratio
cumulative_variance_ratio = np.cumsum(explained_variance_ratio)
ax2.plot(range(1, len(cumulative_variance_ratio)+1,1), cumulative_variance_ratio)

# add a horizontal line at 80% cutoff
ax2.axhline(y=0.8, color='r', linestyle='-')

# find the index where the cumulative variance first exceeds 80%
variance_80 = next(idx for idx, cum_var in enumerate(pca.explained_variance_ratio_.cumsum()) if cum_var > 0.8)

# add a vertical line at the intersection point
ax2.axvline(x=variance_80+1, color='r', linestyle='-')

ax2.set_title('Cumulative Variance Plot')
ax2.set_xlabel('Number of Principal Components')
ax2.set_ylabel('Cumulative Explained Variance Ratio')

# Adjust the layout and spacing between subplots
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
explained_variance_ratio

In [ ]:
# Create a dictionary to store the loadings
loadings = {}

# Iterate through each principal component
for i, pc in enumerate(pca.components_):
    # Create a list to store the loadings of each variable
    loading_list = []
    # Iterate through each variable and its loading
    for j, var in enumerate(num_vars.columns):
        loading = pc[j]
        # Append the variable and its loading to the list
        loading_list.append((var, loading))
    # Sort the list based on the absolute value of the loading
    loading_list.sort(key=lambda x: abs(x[1]), reverse=True)
    # Add the sorted list to the dictionary
    loadings[f'PC{i+1}'] = loading_list

# Print out the loadings
for pc, loading_list in loadings.items():
    print(f'Principal Component {pc}:')
    for var, loading in loading_list:
        print(f'\t{var}: {loading:.3f}')
    print('\n')

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

## Split the dataset into quantitative variables only

# Using predefined quantitative columns, we need 3 PCs to cumulatively explain 80% of the variance
num_vars = students_df_cleaned[quantitative_cols]

# Standardize the data
scaler = StandardScaler()
num_vars_scaled = scaler.fit_transform(num_vars)

# Perform PCA on non-scaled variables
pca = PCA()
pca.fit(num_vars)

# Get the eigenvalues and eigenvectors
eigenvalues = pca.explained_variance_
eigenvectors = pca.components_

# Get the explained variance ratio for each principal component
explained_variance_ratio = pca.explained_variance_ratio_

# Create a figure with custom subplot grid
fig = plt.figure(figsize=(15, 6))
gs = gridspec.GridSpec(1, 3, width_ratios=[0.5, 0.5, 1])
# Create the first subplot for explained variance ratio bar graph
ax1 = plt.subplot(gs[0])
ax1.bar(range(1, len(explained_variance_ratio)+1), explained_variance_ratio)
ax1.set_xticks(range(1, len(explained_variance_ratio)+1))
ax1.set_xlabel('Principal Component')
ax1.set_ylabel('Explained Variance Ratio')
ax1.set_title('Explained Variance Ratio for each Principal Component')

# Create the second subplot for cumulative variance plot
ax2 = plt.subplot(gs[1])
cumulative_variance_ratio = np.cumsum(explained_variance_ratio)
ax2.plot(range(1, len(cumulative_variance_ratio)+1,1), cumulative_variance_ratio)
ax2.axhline(y=0.8, color='r', linestyle='-')
variance_80 = next(idx for idx, cum_var in enumerate(pca.explained_variance_ratio_.cumsum()) if cum_var > 0.8)
ax2.axvline(x=variance_80+1, color='r', linestyle='-')
ax2.set_title('Cumulative Variance Plot')
ax2.set_xlabel('Number of Principal Components')
ax2.set_ylabel('Cumulative Explained Variance Ratio')

# Fit PCA with 3 principal components based on the cumulative variance graph
pca = PCA(n_components=3)
pc_scores = pca.fit_transform(num_vars)

# Define color scheme for each PC
color_scheme = ['steelblue', 'darkorange', 'forestgreen', 'red', 'purple', 'gray', 'teal', 'coral', 'goldenrod', 'pink']

# Create the third subplot for variance explained by each PC for each variable
ax3 = plt.subplot(gs[2])
for i in range(len(pca.explained_variance_ratio_)):
    ax3.bar(num_vars.columns, pca.components_[i], label=f'PC{i+1}', color=color_scheme[i % len(color_scheme)])
ax3.set_title('Variance Explained by first 3 PCs for each Variable')
ax3.set_xlabel('Variables')
ax3.set_ylabel('Variance Explained')
ax3.legend()
ax3.grid(axis='y')
ax3.set_xticklabels(num_vars.columns, rotation=90, ha='right', wrap=True) # Wrap x-axis labels
# Adjust the layout and spacing between subplots
plt.tight_layout()

# Show the plots
plt.show

In [ ]:
# Remove quantitative variables from students_df_cleaned
no_quantitative = students_df_cleaned.drop(quantitative_cols, axis=1)

# Apply PCA to the remaining variables in no_quantitative, keeping only the first 3 principal components
pca = PCA(n_components=3)
pc_scores = pca.fit_transform(no_quantitative)

# Convert pc_scores to a pandas DataFrame
pc_df = pd.DataFrame(pc_scores, columns=['PC1', 'PC2', 'PC3'])

# Concatenate pc_df with no_quantitative
students_df_pca = pd.concat([no_quantitative, pc_df], axis=1)

# View the new dataset
students_df_pca.head()

In [ ]:
# remove NaN values from the DataFrame
students_df_pca = students_df_pca.dropna()

# check for NaN values in the DataFrame
nan_rows = students_df_pca.isnull().any(axis=1)

# print the rows with NaN values
print(students_df_pca[nan_rows])

print(students_df_pca.describe())

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score
from collections import defaultdict

# Getting Data ready for Modelling
X = students_df_cleaned.drop("Target", axis=1)
y = students_df_cleaned.Target

# Create a dictionary of resampling methods and their corresponding datasets
resampling_methods = {'Original': (X, y),
                      'OverSampled': RandomOverSampler().fit_resample(X, y),
                      'UnderSampled': RandomUnderSampler().fit_resample(X, y),
                      'SMOTE': SMOTE().fit_resample(X, y)}

# Create an empty list to store the model results for each resampling method
model_results = []

# Iterate through the resampling methods and their corresponding datasets
for resampling_method, (X_resampled, y_resampled) in resampling_methods.items():
    print('Resampling:', resampling_method)

    # Model 1: Logistic Regression
    logreg_model = LogisticRegression(max_iter=1000)
    logreg_model.fit(X_resampled, y_resampled)
    logreg_scores = cross_val_score(logreg_model, X_resampled, y_resampled, cv=5, scoring='accuracy')
    logreg_precision, logreg_recall, logreg_f1, logreg_support = precision_recall_fscore_support(y_resampled, logreg_model.predict(X_resampled), average='weighted')
    logreg_mean_score = np.mean(logreg_scores)
    model_results.append({'Logistic ' + resampling_method: {'Accuracy': round(logreg_mean_score,4),
                                                                      'Precision': round(logreg_precision,4),
                                                                      'Recall': round(logreg_recall,4),
                                                                      'F1': logreg_f1,
                                                            'model': logreg_model}})

    # Model 2: K-Nearest Neighbors
    knn_model = KNeighborsClassifier()
    knn_model.fit(X_resampled, y_resampled)
    knn_scores = cross_val_score(knn_model, X_resampled, y_resampled, cv=5, scoring='accuracy')
    knn_precision, knn_recall, knn_f1, knn_support = precision_recall_fscore_support(y_resampled, knn_model.predict(X_resampled), average='weighted')
    knn_mean_score = np.mean(knn_scores)
    model_results.append({'KNN ' + resampling_method: {'Accuracy': round(knn_mean_score,4),
                                                                     'Precision': round(knn_precision,4),
                                                                     'Recall': round(knn_recall,4),
                                                                     'F1': knn_f1,
                                                            'model': knn_model}})

    # Model 3: Random Forest
    rf_model = RandomForestClassifier()
    rf_model.fit(X_resampled, y_resampled)
    rf_scores = cross_val_score(rf_model, X_resampled, y_resampled, cv=5, scoring='accuracy')
    rf_precision, rf_recall, rf_f1, rf_support = precision_recall_fscore_support(y_resampled, rf_model.predict(X_resampled), average='weighted')
    rf_mean_score = np.mean(rf_scores)
    model_results.append({'RF ' + resampling_method: {'Accuracy': round(rf_mean_score,4),
                                                                     'Precision': round(rf_precision,4),
                                                                     'Recall': round(rf_recall,4),
                                                                     'F1': rf_scores,
                                                            'model': rf_model}})

    # Model 4: Decision Tree
    dt_model = DecisionTreeClassifier()
    dt_model.fit(X_resampled, y_resampled)
    dt_scores = cross_val_score(dt_model, X_resampled, y_resampled, cv=5, scoring='accuracy')
    dt_precision, dt_recall, dt_f1, dt_support = precision_recall_fscore_support(y_resampled, dt_model.predict(X_resampled), average='weighted')
    dt_mean_score = np.mean(dt_scores)
    model_results.append({'DT ' + resampling_method: {'Accuracy': round(dt_mean_score,4),
                                                                'Precision': round(dt_precision,4),
                                                                'Recall': round(dt_recall,4),
                                                                'F1': dt_scores,
                                                            'model': dt_model}})
    # Model 5: SVM
    svm_model = SVC()
    svm_model.fit(X_resampled, y_resampled)
    svm_scores = cross_val_score(svm_model, X_resampled, y_resampled, cv=5, scoring='accuracy')
    svm_precision, svm_recall, svm_f1, svm_support = precision_recall_fscore_support(y_resampled, svm_model.predict(X_resampled), average='weighted')
    svm_mean_score = np.mean(svm_scores)
    model_results.append({'SVM ' + resampling_method: {'Accuracy': round(svm_mean_score,4),
                                                    'Precision': round(svm_precision,4),
                                                    'Recall': round(svm_recall,4),
                                                    'F1': svm_scores,
                                                    'model': svm_model}})

# Display model performance metrics
for result in model_results:
    for model_name, scores in result.items():
        print(f"\nResults for {model_name}:")
        for metric, value in scores.items():
            print(f"\t{metric}: {value}")

In [ ]:
# Sort the model_results list by accuracy
model_results_sorted = sorted(model_results, key=lambda x: x[list(x.keys())[0]]['Accuracy'], reverse=True)

# Get the model names and their corresponding accuracy and precision scores
model_names = []
accuracy_scores = []
precision_scores = []
for model_dict in model_results_sorted:
    model_name = list(model_dict.keys())[0]
    model_names.append(model_name)
    accuracy_scores.append(model_dict[model_name]['Accuracy'])
    precision_scores.append(model_dict[model_name]['Precision'])

# Plot the accuracy and precision bar charts side-by-side
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(16, 3))

ax1.bar(model_names, accuracy_scores)
ax1.set_ylabel('Accuracy')
ax1.set_title("Accuracy Comparision")
ax1.set_xticks(range(len(model_names)))
ax1.set_xticklabels(model_names, rotation=90)

ax2.bar(model_names, precision_scores)
ax2.set_ylabel('Precision')
ax2.set_title("Precision Comparision")
ax2.set_xticks(range(len(model_names)))
ax2.set_xticklabels(model_names, rotation=90)

fig.suptitle('Model Performance (Cleaned Data with PCA)', fontsize=14)

plt.show()

In [ ]:
# Select the best performing model and its corresponding performance metrics
best_model_name = list(model_results_sorted[0].keys())[0]
best_model_performance = model_results_sorted[0][best_model_name]

# Print the name of the best performing model
print(f"The best performing model is {best_model_name} with the following performance metrics:")

# Print the performance metrics of the best performing model
for metric_name, metric_value in best_model_performance.items():
    if metric_name != 'model':  # exclude the model from the printed performance metrics
        print(f"{metric_name}: {metric_value}")
    else:
        best_model = metric_value

In [ ]:
import numpy as np
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from imblearn.over_sampling import RandomOverSampler

# Create an oversampler (e.g., using SMOTE)
oversampler = RandomOverSampler()  # Your oversampler of choice

rf = best_model

# Create a StratifiedKFold cross-validator with 5 folds
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Initialize an empty list to store confusion matrices for each fold
confusion_matrices = []

# Perform k-fold cross-validation
for train_index, test_index in cv.split(X, y):
    # Split data into training and test sets
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Oversample the training data
    X_train_oversampled, y_train_oversampled = oversampler.fit_resample(X_train, y_train)

    # Train the Random Forest classifier on the oversampled training data
    #rf.fit(X_train_oversampled, y_train_oversampled)

    # Predict labels on the test data
    y_pred = rf.predict(X_test)

    # Compute the confusion matrix for the current fold and append to the list
    confusion_matrix_fold = confusion_matrix(y_test, y_pred)
    confusion_matrices.append(confusion_matrix_fold)
# Print the confusion matrices for each fold
for i, confusion_matrix_fold in enumerate(confusion_matrices):
    print(f"Confusion matrix for fold {i+1}:")
    print(confusion_matrix_fold)
    print()

In [ ]:
# Select the best performing model and it's corresponding accuracy
best_model = model_results_sorted[0][list(model_results_sorted[0].keys())[0]]['model']
best_model_accuracy = model_results_sorted[0][list(model_results_sorted[0].keys())[0]]['Accuracy']

# Print the best performing model's name and accuracy
print(f"The best performing model is {list(model_results_sorted[0].keys())[0]} with accuracy {best_model_accuracy}")

# get feature importances of RandomOverSampler balanced model
feature_importances = best_model.feature_importances_

# create a dataframe of feature importances
importances_df = pd.DataFrame({'feature': X.columns, 'importance': feature_importances})

# sort the dataframe by feature importance in descending order
importances_df = importances_df.sort_values('importance', ascending=False)

# print the top 10 features with their importances
print(importances_df.head(10))

# Set plot size to a square
plt.figure(figsize=(6, 6))

# create a bar chart of feature importances for the top 10 features
top_features = importances_df.head(10)['feature'].values
top_importances = importances_df.head(10)['importance'].values
sorted_idx = top_importances.argsort()
plt.barh(range(len(top_importances)), top_importances[sorted_idx])
plt.yticks(range(len(top_importances)), top_features[sorted_idx], wrap=True) # Wrap text
plt.title("Random Forest (Oversampled)\nwith Cleaned Data")
plt.xlabel('Feature Importance')
plt.show()

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from sklearn.metrics import precision_recall_fscore_support
from collections import defaultdict

# Getting Data ready for Modelling
X = students_df_pca.drop("Target", axis=1)
y = students_df_pca.Target

# Create a dictionary of resampling methods and their corresponding datasets
resampling_methods = {'Original': (X, y),
                      'OverSampled': RandomOverSampler().fit_resample(X, y),
                      'UnderSampled': RandomUnderSampler().fit_resample(X, y),
                      'SMOTE': SMOTE().fit_resample(X, y)}

# Create an empty list to store the model results for each resampling method
model_results = []

# Iterate through the resampling methods and their corresponding datasets
for resampling_method, (X_resampled, y_resampled) in resampling_methods.items():
    print('Resampling:', resampling_method)

    # Model 1: Logistic Regression
    logreg_model = LogisticRegression(max_iter=1000)
    logreg_model.fit(X_resampled, y_resampled)
    logreg_scores = cross_val_score(logreg_model, X_resampled, y_resampled, cv=5, scoring='accuracy')
    logreg_precision, logreg_recall, logreg_f1, logreg_support = precision_recall_fscore_support(y_resampled, logreg_model.predict(X_resampled), average='weighted')
    logreg_mean_score = np.mean(logreg_scores)
    model_results.append({'Logistic ' + resampling_method: {'Accuracy': round(logreg_mean_score,2),
                                                                      'Precision': round(logreg_precision,2),
                                                                      'Recall': round(logreg_recall,2),
                                                                      'F1': logreg_f1,
                                                            'model': logreg_model}})

    # Model 2: K-Nearest Neighbors
    knn_model = KNeighborsClassifier()
    knn_model.fit(X_resampled, y_resampled)
    knn_scores = cross_val_score(knn_model, X_resampled, y_resampled, cv=5, scoring='accuracy')
    knn_precision, knn_recall, knn_f1, knn_support = precision_recall_fscore_support(y_resampled, knn_model.predict(X_resampled), average='weighted')
    knn_mean_score = np.mean(knn_scores)
    model_results.append({'KNN ' + resampling_method: {'Accuracy': round(knn_mean_score,2),
                                                                     'Precision': round(knn_precision,2),
                                                                     'Recall': round(knn_recall,2),
                                                                     'F1': knn_f1,
                                                            'model': knn_model}})

    # Model 3: Random Forest
    rf_model = RandomForestClassifier()
    rf_model.fit(X_resampled, y_resampled)
    rf_scores = cross_val_score(rf_model, X_resampled, y_resampled, cv=5, scoring='accuracy')
    rf_precision, rf_recall, rf_f1, rf_support = precision_recall_fscore_support(y_resampled, rf_model.predict(X_resampled), average='weighted')
    rf_mean_score = np.mean(rf_scores)
    model_results.append({'RF ' + resampling_method: {'Accuracy': round(rf_mean_score,2),
                                                                     'Precision': round(rf_precision,2),
                                                                     'Recall': round(rf_recall,2),
                                                                     'F1': rf_scores,
                                                            'model': rf_model}})
    # Model 4: Decision Tree
    dt_model = DecisionTreeClassifier()
    dt_model.fit(X_resampled, y_resampled)
    dt_scores = cross_val_score(dt_model, X_resampled, y_resampled, cv=5, scoring='accuracy')
    dt_precision, dt_recall, dt_f1, dt_support = precision_recall_fscore_support(y_resampled, dt_model.predict(X_resampled), average='weighted')
    dt_mean_score = np.mean(dt_scores)
    model_results.append({'DT ' + resampling_method: {'Accuracy': round(dt_mean_score,2),
                                                                'Precision': round(dt_precision,2),
                                                                'Recall': round(dt_recall,2),
                                                                'F1': dt_scores,
                                                            'model': dt_model}})
    # Model 5: SVM
    svm_model = SVC()
    svm_model.fit(X_resampled, y_resampled)
    svm_scores = cross_val_score(svm_model, X_resampled, y_resampled, cv=5, scoring='accuracy')
    svm_precision, svm_recall, svm_f1, svm_support = precision_recall_fscore_support(y_resampled, svm_model.predict(X_resampled), average='weighted')
    svm_mean_score = np.mean(svm_scores)
    model_results.append({'SVM ' + resampling_method: {'Accuracy': round(svm_mean_score,2),
                                                    'Precision': round(svm_precision,2),
                                                    'Recall': round(svm_recall,2),
                                                    'F1': svm_scores,
                                                    'model': svm_model}})

# Display model performance metrics
for result in model_results:
    for model_name, scores in result.items():
        print(f"\nResults for {model_name}:")
        for metric, value in scores.items():
            print(f"\t{metric}: {value}")

In [ ]:
# Sort the model_results list by accuracy
model_results_sorted = sorted(model_results, key=lambda x: x[list(x.keys())[0]]['Accuracy'], reverse=True)

# Get the model names and their corresponding accuracy and precision scores
model_names = []
accuracy_scores = []
precision_scores = []
for model_dict in model_results_sorted:
    model_name = list(model_dict.keys())[0]
    model_names.append(model_name)
    accuracy_scores.append(model_dict[model_name]['Accuracy'])
    precision_scores.append(model_dict[model_name]['Precision'])

# Plot the accuracy and precision bar charts side-by-side
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(16, 3))

ax1.bar(model_names, accuracy_scores)
ax1.set_ylabel('Accuracy')
ax1.set_title("Accuracy Comparision")
ax1.set_xticks(range(len(model_names)))
ax1.set_xticklabels(model_names, rotation=90)

ax2.bar(model_names, precision_scores)
ax2.set_ylabel('Precision')
ax2.set_title("Precision Comparision")
ax2.set_xticks(range(len(model_names)))
ax2.set_xticklabels(model_names, rotation=90)

fig.suptitle('Model Performance (Cleaned Data with PCA)', fontsize=14)

plt.show()

In [ ]:
# Select the best performing model and its corresponding performance metrics
best_model_name = list(model_results_sorted[0].keys())[0]
best_model_performance = model_results_sorted[0][best_model_name]

# Print the name of the best performing model
print(f"The best performing model is {best_model_name} with the following performance metrics:")

# Print the performance metrics of the best performing model
for metric_name, metric_value in best_model_performance.items():
    if metric_name != 'model':  # exclude the model from the printed performance metrics
        print(f"{metric_name}: {metric_value}")

In [ ]:
# Select the best performing model and it's corresponding accuracy
best_model = model_results_sorted[0][list(model_results_sorted[0].keys())[0]]['model']
best_model_accuracy = model_results_sorted[0][list(model_results_sorted[0].keys())[0]]['Accuracy']

# Print the best performing model's name and accuracy
print(f"The best performing model is {list(model_results_sorted[0].keys())[0]} with accuracy {best_model_accuracy}")

# get feature importances of RandomOverSampler balanced model
feature_importances = best_model.feature_importances_

# create a dataframe of feature importances
importances_df = pd.DataFrame({'feature': X.columns, 'importance': feature_importances})

# sort the dataframe by feature importance in descending order
importances_df = importances_df.sort_values('importance', ascending=False)

# print the top 10 features with their importances
print(importances_df.head(10))

# Set plot size to a square
plt.figure(figsize=(6, 6))

# create a bar chart of feature importances for the top 10 features
top_features = importances_df.head(10)['feature'].values
top_importances = importances_df.head(10)['importance'].values
sorted_idx = top_importances.argsort()
plt.barh(range(len(top_importances)), top_importances[sorted_idx])
plt.yticks(range(len(top_importances)), top_features[sorted_idx], wrap=True) # Wrap text
plt.title("Random Forest (Oversampled)\nwith Cleaned Data")
plt.xlabel('Feature Importance')
plt.show()

In [ ]:
# Convert Target to numerical
students_df_normalised_no_outliers["Target"].replace('Dropout', 0, inplace=True)
students_df_normalised_no_outliers["Target"].replace('Graduate', 1, inplace=True)
students_df_normalised_no_outliers["Target"].replace('Enrolled', 2, inplace=True)

# Removing enrolled from Target
students_df_normalised_no_outliers = students_df_normalised_no_outliers[students_df_normalised_no_outliers['Target'] != 2]

# Define features to remove
features_to_remove = ["Curricular units 1st sem (credited)", "Curricular units 1st sem (enrolled)",
                      "Curricular units 1st sem (evaluations)", "Curricular units 1st sem (approved)",
                      "Curricular units 1st sem (grade)", "Curricular units 2nd sem (approved)", "Nationality"]

# remove the specified columns from the dataframe
students_df_normalised_no_outliers = students_df_normalised_no_outliers.drop(features_to_remove, axis=1)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from sklearn.metrics import precision_recall_fscore_support
from collections import defaultdict

# Getting Data ready for Modelling
X = students_df_normalised_no_outliers.drop("Target", axis=1)
y = students_df_normalised_no_outliers.Target

# Create a dictionary of resampling methods and their corresponding datasets
resampling_methods = {'Original': (X, y),
                      'OverSampled': RandomOverSampler().fit_resample(X, y),
                      'UnderSampled': RandomUnderSampler().fit_resample(X, y),
                      'SMOTE': SMOTE().fit_resample(X, y)}

# Create an empty list to store the model results for each resampling method
model_results_2 = []

# Iterate through the resampling methods and their corresponding datasets
for resampling_method, (X_resampled, y_resampled) in resampling_methods.items():
    print('Resampling:', resampling_method)


    # Model 1: Logistic Regression
    logreg_model = LogisticRegression(max_iter=1000)
    logreg_model.fit(X_resampled, y_resampled)
    logreg_scores = cross_val_score(logreg_model, X_resampled, y_resampled, cv=5, scoring='accuracy')
    logreg_precision, logreg_recall, logreg_f1, logreg_support = precision_recall_fscore_support(y_resampled, logreg_model.predict(X_resampled), average='weighted')
    logreg_mean_score = np.mean(logreg_scores)
    model_results_2.append({'Logistic ' + resampling_method: {'Accuracy': round(logreg_mean_score,2),
                                                                      'Precision': round(logreg_precision,2),
                                                                      'Recall': round(logreg_recall,2),
                                                                      'F1': logreg_f1,
                                                            'model': logreg_model}})

    # Model 2: K-Nearest Neighbors
    knn_model = KNeighborsClassifier()
    knn_model.fit(X_resampled, y_resampled)
    knn_scores = cross_val_score(knn_model, X_resampled, y_resampled, cv=5, scoring='accuracy')
    knn_precision, knn_recall, knn_f1, knn_support = precision_recall_fscore_support(y_resampled, knn_model.predict(X_resampled), average='weighted')
    knn_mean_score = np.mean(knn_scores)
    model_results_2.append({'KNN ' + resampling_method: {'Accuracy': round(knn_mean_score,2),
                                                                     'Precision': round(knn_precision,2),
                                                                     'Recall': round(knn_recall,2),
                                                                     'F1': knn_f1,
                                                            'model': knn_model}})

    # Model 3: Random Forest
    rf_model = RandomForestClassifier()
    rf_model.fit(X_resampled, y_resampled)
    rf_scores = cross_val_score(rf_model, X_resampled, y_resampled, cv=5, scoring='accuracy')
    rf_precision, rf_recall, rf_f1, rf_support = precision_recall_fscore_support(y_resampled, rf_model.predict(X_resampled), average='weighted')
    rf_mean_score = np.mean(rf_scores)
    model_results_2.append({'RF ' + resampling_method: {'Accuracy': round(rf_mean_score,2),
                                                                     'Precision': round(rf_precision,2),
                                                                     'Recall': round(rf_recall,2),
                                                                     'F1': rf_scores,
                                                            'model': rf_model}})
    # Model 4: Decision Tree
    dt_model = DecisionTreeClassifier()
    dt_model.fit(X_resampled, y_resampled)
    dt_scores = cross_val_score(dt_model, X_resampled, y_resampled, cv=5, scoring='accuracy')
    dt_precision, dt_recall, dt_f1, dt_support = precision_recall_fscore_support(y_resampled, dt_model.predict(X_resampled), average='weighted')
    dt_mean_score = np.mean(dt_scores)
    model_results_2.append({'DT ' + resampling_method: {'Accuracy': round(dt_mean_score,2),
                                                                'Precision': round(dt_precision,2),
                                                                'Recall': round(dt_recall,2),
                                                                'F1': dt_scores,
                                                            'model': dt_model}})
    # Model 5: SVM
    svm_model = SVC()
    svm_model.fit(X_resampled, y_resampled)
    svm_scores = cross_val_score(svm_model, X_resampled, y_resampled, cv=5, scoring='accuracy')
    svm_precision, svm_recall, svm_f1, svm_support = precision_recall_fscore_support(y_resampled, svm_model.predict(X_resampled), average='weighted')
    svm_mean_score = np.mean(svm_scores)
    model_results_2.append({'SVM ' + resampling_method: {'Accuracy': round(svm_mean_score,2),
                                                    'Precision': round(svm_precision,2),
                                                    'Recall': round(svm_recall,2),
                                                    'F1': svm_scores,
                                                    'model': svm_model}})

# Display model performance metrics
for result in model_results_2:
    for model_name, scores in result.items():
        print(f"\nResults for {model_name}:")
        for metric, value in scores.items():
            print(f"\t{metric}: {value}")

In [ ]:
# Sort the model_results list by accuracy
model_results_sorted_2 = sorted(model_results_2, key=lambda x: x[list(x.keys())[0]]['Accuracy'], reverse=True)

# Get the model names and their corresponding accuracy and precision scores
model_names = []
accuracy_scores = []
precision_scores = []
for model_dict in model_results_sorted_2:
    model_name = list(model_dict.keys())[0]
    model_names.append(model_name)
    accuracy_scores.append(model_dict[model_name]['Accuracy'])
    precision_scores.append(model_dict[model_name]['Precision'])

# Plot the accuracy and precision bar charts side-by-side
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(16, 3))

ax1.bar(model_names, accuracy_scores)
ax1.set_ylabel('Accuracy')
ax1.set_title("Accuracy Comparision", fontsize=11)
ax1.set_xticks(range(len(model_names)))
ax1.set_xticklabels(model_names, rotation=90)

ax2.bar(model_names, precision_scores)
ax2.set_ylabel('Precision')
ax2.set_title("Precision Comparision", fontsize=11)
ax2.set_xticks(range(len(model_names)))
ax2.set_xticklabels(model_names, rotation=90)

fig.suptitle('Model Performance (Normalized, No Outliers)', fontsize=14)

plt.show()

In [ ]:
# Select the best performing model and its corresponding performance metrics
best_model_name = list(model_results_sorted_2[0].keys())[0]
best_model_performance = model_results_sorted_2[0][best_model_name]

# Print the name of the best performing model
print(f"The best performing model is {best_model_name} with the following performance metrics:")

# Print the performance metrics of the best performing model
for metric_name, metric_value in best_model_performance.items():
    if metric_name != 'model':  # exclude the model from the printed performance metrics
        print(f"{metric_name}: {metric_value}")


In [ ]:
# Select the best performing model and it's corresponding accuracy
best_model = model_results_sorted_2[0][list(model_results_sorted_2[0].keys())[0]]['model']
best_model_accuracy = model_results_sorted_2[0][list(model_results_sorted_2[0].keys())[0]]['Accuracy']

# Print the best performing model's name and accuracy
print(f"The best performing model is {list(model_results_sorted_2[0].keys())[0]} with accuracy {best_model_accuracy}")

# get feature importances of RandomOverSampler balanced model
feature_importances = best_model.feature_importances_

# create a dataframe of feature importances
importances_df = pd.DataFrame({'feature': X.columns, 'importance': feature_importances})

# sort the dataframe by feature importance in descending order
importances_df = importances_df.sort_values('importance', ascending=False)

# print the top 10 features with their importances
print(importances_df.head(10))

# Set plot size to a square
plt.figure(figsize=(6, 6))

# Create a bar chart of feature importances for the top 10 features
top_features = importances_df.head(10)['feature'].values
top_importances = importances_df.head(10)['importance'].values
sorted_idx = top_importances.argsort()
plt.barh(range(len(top_importances)), top_importances[sorted_idx])
plt.yticks(range(len(top_importances)), top_features[sorted_idx], wrap=True) # Wrap text
plt.title("Random Forest (Oversampled)\nwith Additionally Normalised and Outliers Removed Data")
plt.xlabel('Feature Importance')

plt.show()